In [7]:
import torch
import torch.nn.functional as F
from gpt_model import GPT, GPTConfig
import tiktoken

# Force CPU usage
device = "cpu"
print(f"Using device: {device}")

# Load the model
checkpoint_path = "log124M_10B/model_19072.pt"
checkpoint = torch.load(checkpoint_path, map_location=device)
config = checkpoint['config']
model = GPT(config)
model.load_state_dict(checkpoint['model'])
model.to(device)
model.eval()

# Set up the tokenizer
enc = tiktoken.get_encoding("gpt2")

Using device: cpu


/tmp/ipykernel_268119/57045122.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


In [8]:
import torch
import torch.nn.functional as F
from gpt_json import generate_json

numeric_tokens = [i for i in range(enc.n_vocab) if enc.decode([i]).isdigit()]
period_token = enc.encode(".")[0]
comma_token = enc.encode(",")[0]
numeric_tokens.append(period_token)
numeric_tokens.append(comma_token)
nt = torch.tensor(numeric_tokens, dtype=torch.long).unsqueeze(0).to(device)

# Example usage:
schema = {
        "name": {"type": "string"},
        "age": {"type": "number"}
}

prompt_GPT = f"Generate an example person caled Tom!!!! and output result in the following JSON schema format:\n{schema}\n"
print(prompt_GPT)
result_GPT = generate_json(prompt_GPT, 
                           schema, 
                           model, 
                           enc, 
                           device, 
                           nt, 
                           period_token, 
                           comma_token,
                           seed=0)
print(result_GPT)

Generate an example person caled Tom!!!! and output result in the following JSON schema format:
{'name': {'type': 'string'}, 'age': {'type': 'number'}}

Generate an example person caled Tom!!!! and output result in the following JSON schema format:
{'name': {'type': 'string'}, 'age': {'type': 'number'}}
{"name": "Name","age": 1987}


In [9]:
## Test Jsonformer
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading model and tokenizer...")
model_name = "databricks/dolly-v2-3b"

# Specify CPU as the device
device = torch.device("cpu")

# Load the model to CPU
model_json = AutoModelForCausalLM.from_pretrained(
    model_name, 
    use_cache=True, 
    device_map="cpu",  # This ensures CPU usage
    torch_dtype=torch.float32  # Use float32 for CPU
)

# Explicitly move the model to CPU (though it should already be there)
model_json = model_json.to(device)

# Load the tokenizer (tokenizer is not affected by device choice)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)

print("Loaded model and tokenizer")

# Verify that the model is on CPU
print(f"Model is on: {next(model_json.parameters()).device}")

Loading model and tokenizer...
Loaded model and tokenizer
Model is on: cpu


/home/ztkpat001/repos/build-nanogpt/myenv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer


person = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
    }
}

builder = Jsonformer(
    model=model_json,
    tokenizer=tokenizer,
    json_schema=person,
    prompt="Generate details of a person who is a student",
    debug=False,
)

print("Generating...")
output = builder()

highlight_values(output)


Generating...
{
  name: "John Doe",
  age: 30.0,
  is_student: True
}


In [11]:
## Jsonformer
print(f"Jsonformer: {isinstance(output, dict)}")

Jsonformer: True


In [12]:
## GPT-2
import json
json_string = result_GPT[len(prompt_GPT):]
print(f"GPT-2: {json_string}")
json.loads(result_GPT[len(prompt_GPT):])
# It works!!

GPT-2: {"name": "Name","age": 1987}


{'name': 'Name', 'age': 1987}

### Post-mortem
- Really bad LLMs (i.e. in the way GPT-2 is worse than GPT-3) are better for testing since they are less likely to "obey" JSON schema, or any other structure. So your code has to be more robust to account for it. For example, GPT-4 might not need any of the additional code I wrote to handle the LLMs' output.
- You can enforce any output structure in the same way I have done here.
- Batch processing output could is more complicated as you may have the output sequences building at different rates. 